In [1]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import time
from datetime import datetime, timezone

In [4]:
%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_seq_items', 500)
pd.set_option("display.max_colwidth", None)

# Selection

In [5]:
df_ss = pd.read_excel("data/stock_stats.xlsx", sheet_name="stock_stats")

In [6]:
print("Total stocks: ", len(df_ss))
print("Total columns ", len(df_ss.columns))
df_ss.columns

Total stocks:  9000
Total columns  148


Index(['52WeekChange', 'SandP52WeekChange', 'address1', 'algorithm', 'ask',
       'askSize', 'averageDailyVolume10Day', 'averageVolume',
       'averageVolume10days', 'beta', 'bid', 'bidSize', 'bookValue',
       'category', 'city', 'coinMarketCapLink', 'companyOfficers', 'country',
       'currency', 'currencySymbol', 'currentPrice', 'currentRatio',
       'dateShortInterest', 'dayHigh', 'dayLow', 'debtToEquity',
       'dividendDate', 'dividendRate', 'dividendYield', 'earnings',
       'earningsGrowth', 'earningsQuarterlyGrowth', 'ebitda', 'ebitdaMargins',
       'enterpriseToEbitda', 'enterpriseToRevenue', 'enterpriseValue',
       'exDividendDate', 'exchange', 'exchangeDataDelayedBy', 'exchangeName',
       'fax', 'fiftyDayAverage', 'fiftyTwoWeekHigh', 'fiftyTwoWeekLow',
       'financialCurrency', 'firstTradeDateEpochUtc',
       'fiveYearAvgDividendYield', 'floatShares', 'forwardEps', 'forwardPE',
       'freeCashflow', 'fromCurrency', 'fullTimeEmployees', 'fundFamily',
       '

In [7]:
df_ss[df_ss["symbol"] == "AAPL"].head(1).T

,20
52WeekChange,0.074196
SandP52WeekChange,0.003393
address1,One Apple Park Way
algorithm,NaN
ask,169.59
askSize,1000.0
averageDailyVolume10Day,50620170.0
averageVolume,62305159.0
averageVolume10days,50620170.0
beta,1.296885


In [8]:
current_year = datetime.now().year

def from_epoch_time(value) -> datetime:
    return pd.to_datetime(value, unit="s")

df_ss.rename(columns=str.lower, inplace=True)
df_ss["lastdividenddate"] = df_ss["lastdividenddate"].apply(from_epoch_time)

INFO_FIELDS = [ "symbol", 
                "shortname", 
                "beta",
                "currentprice",
                "debttoequity", 
                "dividendrate", 
                "dividendyield", 
                "earnings",
                "exdividenddate",
                "fiveyearavgdividendyield",
                "freecashflow",
                "lastdividenddate", 
                "lastdividendvalue",
                "pegratio",
                "pricetobook",
                "sector",
                "trailingannualdividendyield",
                "trailingpe",
              ]

In [9]:
from scipy.optimize import minimize

# Read the data from the list of stocks
stocks = df_ss.copy()

# Replace infinity with NaN
stocks.replace([np.inf, -np.inf], np.nan, inplace=True)

In [10]:
# -- Five year dividend yield > median x 1.5 by sector
CRITERIA_FLD_HIST_DIV_YIELD_SECTOR = "criteria_hist_div_yield_sector"

def check_hist_div_yield_sector(df):
    return df['sector'].map(df.groupby('sector')['fiveyearavgdividendyield'].median()) * 1.2

criteria_hist_div_yield_sector = lambda df:  (df['fiveyearavgdividendyield'] > df[CRITERIA_FLD_HIST_DIV_YIELD_SECTOR])

# -- Five year dividend yield > minimum expected
CRITERIA_FLD_HIST_DIV_YIELD_MIN = "criteria_hist_div_yield_min"

def check_hist_div_yield_min():
    return 13

criteria_hist_div_yield_min = lambda df: df['fiveyearavgdividendyield'] > df[CRITERIA_FLD_HIST_DIV_YIELD_MIN]

# -- Dividend yield > minimum expected
CRITERIA_FLD_DIV_YIELD_MIN = "criteria_div_yield_min"

def check_div_yield_min():
    return 0.13

criteria_div_yield_min = lambda df: df['dividendyield'] > df[CRITERIA_FLD_DIV_YIELD_MIN]

# -- Pay dividend in the last year
CRITERIA_FLD_DIV_YEAR = "criteria_div_year"

def check_div_year(df):
    return  df['lastdividenddate'].dt.year.astype("Int64", errors="ignore")

criteria_div_year = lambda df:  df['criteria_div_year'] >= (current_year - 1)

# -- Sector is not blank
CRITERIA_FLD_SECTOR = "criteria_sector"

def check_sector(df):
    return ~df['sector'].isnull()

criteria_sector = lambda df: df[CRITERIA_FLD_SECTOR]

# -- Trailing PE
CRITERIA_FLD_PE = "criteria_pe"

def check_pe(df):
    return  df['sector'].map(df.groupby('sector')['trailingpe'].median()) * 1.0

criteria_pe = lambda df: df['trailingpe'] < df[CRITERIA_FLD_PE]

# -- P/B ratio
CRITERIA_FLD_PB = "criteria_pb"

def check_pb(df):
    return  df['sector'].map(df.groupby('sector')['pricetobook'].median()) * 1.0

criteria_pb = lambda df: df['pricetobook'] < df[CRITERIA_FLD_PB]

# -- Debt to equity ratio
CRITERIA_FLD_DE = "criteria_de"

def check_de(df):
    return  df['sector'].map(df.groupby('sector')['debttoequity'].median()) * 1.0

criteria_de = lambda df: df['debttoequity'] < df[CRITERIA_FLD_DE]

criteria = {
    #CRITERIA_FLD_HIST_DIV_YIELD_SECTOR: criteria_hist_div_yield_sector,
    #CRITERIA_FLD_HIST_DIV_YIELD_MIN: criteria_hist_div_yield_min,
    CRITERIA_FLD_SECTOR: criteria_sector,
    CRITERIA_FLD_DIV_YEAR: criteria_div_year,
    CRITERIA_FLD_DIV_YIELD_MIN: criteria_div_yield_min,
    CRITERIA_FLD_PE: criteria_pe,
    CRITERIA_FLD_PB: criteria_pb,
    CRITERIA_FLD_DE: criteria_de,
}

CRITERIA_FIELDS = [
                   CRITERIA_FLD_HIST_DIV_YIELD_SECTOR,
                   CRITERIA_FLD_HIST_DIV_YIELD_MIN,
                   CRITERIA_FLD_DIV_YIELD_MIN,
                   CRITERIA_FLD_DIV_YEAR,
                   CRITERIA_FLD_SECTOR,
                   CRITERIA_FLD_PE,
                   CRITERIA_FLD_PB,
                   CRITERIA_FLD_DE
                ]

stocks[CRITERIA_FLD_HIST_DIV_YIELD_SECTOR] = check_hist_div_yield_sector(stocks)
stocks[CRITERIA_FLD_HIST_DIV_YIELD_MIN] = check_hist_div_yield_min()
stocks[CRITERIA_FLD_DIV_YIELD_MIN] = check_div_yield_min()
stocks[CRITERIA_FLD_DIV_YEAR] = check_div_year(stocks)
stocks[CRITERIA_FLD_SECTOR] = check_sector(stocks)
stocks[CRITERIA_FLD_PE] = check_pe(stocks)
stocks[CRITERIA_FLD_PB] = check_pb(stocks)
stocks[CRITERIA_FLD_DE] = check_de(stocks)

value_stocks = stocks.copy()
for name in criteria.keys():
    value_stocks = value_stocks.loc[criteria[name]]

print(len(value_stocks))
value_stocks[CRITERIA_FIELDS + INFO_FIELDS].head(100).T

11


,210,1267,2602,3474,3578,3603,4580,4877,5194,6244,7127
criteria_hist_div_yield_sector,7.044,7.818,2.034,2.472,2.034,2.034,4.14,3.012,4.86,7.044,2.034
criteria_hist_div_yield_min,13,13,13,13,13,13,13,13,13,13,13
criteria_div_yield_min,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13
criteria_div_year,2023,2023,2023,2023,2023,2023,2023,2022,2023,2023,2023
criteria_sector,True,True,True,True,True,True,True,True,True,True,True
criteria_pe,21.258333,6.692771,16.070652,12.225921,16.070652,16.070652,18.41784,19.924421,9.603025,21.258333,16.070652
criteria_pb,1.111662,1.316917,1.654941,1.458235,1.654941,1.654941,1.520059,1.555158,0.923088,1.111662,1.654941
criteria_de,129.6925,59.9145,68.814,40.785,68.814,68.814,105.521,58.624,58.027,129.6925,68.814
symbol,AFCG,BRY,EGLE,GGB,GNK,GOGL,KEN,LND,MGU,PDM,SBLK
shortname,"AFC Gamma, Inc.",Berry Corporation (bry),Eagle Bulk Shipping Inc.,Gerdau S.A.,Genco Shipping & Trading Limite,Golden Ocean Group Limited,Kenon Holdings Ltd.,Brasilagro Brazilian Agric Real,Macquarie Global Infrastructure,"Piedmont Office Realty Trust, I",Star Bulk Carriers Corp.


In [11]:
def objective(weights, stocks):
    returns = np.array([stock['dividendyield'] for stock in stocks])
    cov_matrix = np.cov(returns)
    portfolio_return = np.dot(weights, returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return -portfolio_return / portfolio_volatility

In [12]:
def optimize_portfolio(stocks):
    sectors = set([stock['sector'] for stock in stocks])
    num_sectors = len(sectors)
    num_stocks = len(stocks)
    
    def sum_constraint(x):
        return np.sum(x) - 1.0
    
    constraints = [
        {'type': 'eq', 'fun': sum_constraint}
    ]
    
    bounds = [(0, 1) for i in range(num_stocks)]
    
    result = minimize(objective, np.ones(num_stocks) / num_stocks, args=(stocks,), method='SLSQP', bounds=bounds, constraints=constraints)
    
    return result.x

shortlisted_stocks = value_stocks[INFO_FIELDS].to_dict("records")
portfolio_weights = optimize_portfolio(shortlisted_stocks)
#print(shortlisted_stocks)
print(portfolio_weights)

[0.081742   0.10023051 0.06269114 0.0606282  0.07270295 0.07752031
 0.2130546  0.09126677 0.07773821 0.05852164 0.10390367]


In [13]:
sectors = set([stock['sector'] for stock in shortlisted_stocks])
print(sectors)

{'Utilities', 'Industrials', 'Basic Materials', 'Consumer Defensive', 'Financial Services', 'Energy', 'Real Estate'}


In [27]:
AMT_INVEST = 1000
total_amt = 0
counter = 0
for stock in shortlisted_stocks:
    amt = portfolio_weights[counter] * AMT_INVEST
    total_amt = total_amt + amt
    counter = counter + 1
    #print(f"{stock['symbol']},{stock['shortname']} - {round(amt,2)}")
    print(f"{stock['symbol']},{stock['shortname']}")

print("Total amt: ", total_amt)

AFCG,AFC Gamma, Inc.
BRY,Berry Corporation (bry)
EGLE,Eagle Bulk Shipping Inc.
GGB,Gerdau S.A.
GNK,Genco Shipping & Trading Limite
GOGL,Golden Ocean Group Limited
KEN,Kenon Holdings Ltd.
LND,Brasilagro Brazilian Agric Real
MGU,Macquarie Global Infrastructure
PDM,Piedmont Office Realty Trust, I
SBLK,Star Bulk Carriers Corp.
Total amt:  1000.0000000000001


In [25]:
print(value_stocks['dividendyield'].sum())

2.32529999


In [22]:
print(0.4954/2.325 * 1000)

213.0752688172043


Stocks to track
- PBR-A
- PBR
- GOGL
- ORC - monthly
- OXLC - monthly

## Piotroski Score

In [1]:
# https://python.plainenglish.io/finding-the-best-value-stock-with-piotroski-score-in-python-5a793580226b